In [ ]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
import matplotlib.pyplot as plt
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler



In [ ]:
X = np.load('Week1/initial_data/function_3/initial_inputs.npy')
y = np.load('Week1/initial_data/function_3/initial_outputs.npy')
print(f"X is:{X}")
print(f"Y is :{y}")

In [ ]:
# add the week 1 input and  result to the array
X = np.append(X, [[0.489796,0.612245,0.326531]], axis=0)
y = np.append(y, [-0.04258343], axis=0)

# add the week 2 input and result to the array
X = np.append(X, [[0.428571, 0.448980, 0.489796]], axis=0)
y = np.append(y, [0.-0.007164582542736379], axis=0)

# add the week 3 input and result to the array
X = np.append(X, [[0.545455, 0.666667, 0.585859]], axis=0)
y = np.append(y, [-0.047753580848788356], axis=0)

# add the week 4 input and result to the array
X = np.append(X, [[0.474657, 0.469540, 0.376028]], axis=0)
y = np.append(y, [-0.016404189239675314], axis=0)

# add the week 5 input and result to the array
X = np.append(X, [[0.470795, 0.469432, 0.384002]], axis=0)
y = np.append(y, [-0.0457763720690547], axis=0)

print(f"X is:{X}")
print(f"Y is :{y}")

In [ ]:
print(f"max y is :{np.max(y)}")

In [ ]:
# Define GP model
kernel = Matern(nu=2.5, length_scale_bounds=(1e-2, 1e2))
gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-10, normalize_y=True)
gp.fit(X, y)

In [ ]:
# UCB acquisition function
# week 6 changed kappa to 4.0 from 2.0
def ucb(x, gp, kappa=4.0):
    x = x.reshape(1, -1)
    mu, sigma = gp.predict(x, return_std=True)
    return mu + kappa * sigma

In [ ]:
grid_size = 100
x1 = np.linspace(0, 1, grid_size)
x2 = np.linspace(0, 1, grid_size)
x3 = np.linspace(0, 1, grid_size)
X_grid = np.array([[a, b, c] for a in x1 for b in x2 for c in x3])

In [ ]:
ucb_values = np.array([ucb(x, gp) for x in X_grid])


In [ ]:
best_idx = np.argmax(ucb_values)
next_best_point = X_grid[best_idx]
pred_mean, pred_std = gp.predict(next_best_point.reshape(1, -1), return_std=True)
print("Next best point to evaluate (UCB):", np.round(next_best_point,6))
print("Next best point to evaluate (UCB) without rounding:", next_best_point)
print("Predicted maximum value (mean):", pred_mean[0])


In [ ]:
# Scale inputs
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train SVM surrogate
svm_surrogate = SVR(kernel='rbf', C=10.0, epsilon=0.1)
svm_surrogate.fit(X_scaled, y)

# Predict on new candidates
X_new = np.random.rand(10000000, 3)
X_new_scaled = scaler.transform(X_new)
y_pred = svm_surrogate.predict(X_new_scaled)

# Maximization: find best candidate
best_idx = np.argmax(y_pred)
best_candidate = X_new[best_idx]
best_predicted_value = y_pred[best_idx]

print("Best candidate input:", best_candidate)
print("Best candidate input rounded:", np.round(best_candidate, 6))
print("Predicted maximum value:", best_predicted_value)